In [5]:
import os
import json
import pandas as pd
import pysrt
import re

In [6]:
def compute_iou(interval_1, interval_2):
    start_i, end_i = float(interval_1[0]), float(interval_1[1])
    start, end = float(interval_2[0]), float(interval_2[1])
    intersection = max(0, min(end, end_i) - max(start, start_i))
    union = min(max(end, end_i) - min(start, start_i), end-start + end_i-start_i)
    iou = float(intersection) / (union + 1e-8)
    return iou

In [7]:
def save_json(content, save_path):
    with open(save_path, 'w') as f:
        f.write(json.dumps(content))
def load_jsonl(filename):
    with open(filename, "r") as f:
        return [json.loads(l.strip("\n")) for l in f.readlines()]
def load_result_json(filename):
    with open(filename, "r") as f:
        data = json.load(f)
    #print(data)
    return data

BASELINE Evaluation

In [11]:
val = load_jsonl(f'/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa/all_data_val.json')
test = load_jsonl(f'/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa/all_data_test.json')

In [20]:
folders = ['/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa', 
           '/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa_finetuned_versions/tvqa_finetuned_on_visual_med', 
           '/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa_finetuned_versions/tvqa_finetuned_on_visual_nonmed', 
           '/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa_finetuned_versions/tvqa_finetuned_on_visual_med_and_gen_audio']

In [13]:
test_med_queries_path = "/home/hlpark/shared/TVQA/output_gpt_tvqa_test_queries.txt"
val_med_queries_path = "/home/hlpark/shared/TVQA/output_gpt_tvqa_val_queries.txt"
test_med_query_f_metamap_list, val_med_query_f_metamap_list = [], []
with open(test_med_queries_path, "r") as f:
    lines = f.read()
    for line in lines.split("\n"):
        if line == "\n" or line == "":
            continue
        if line.startswith(" ") or line.startswith("-"):
            line = line.lstrip("-").lstrip(" ")
        test_med_query_f_metamap_list.append(line.lower().replace(" ", "").replace("?", "").replace("\n", "").replace("-", "").replace("'", "").replace("\"", "").replace(",", "").replace(".", "").replace("/", "").replace(">", ""))
f.close()
with open(val_med_queries_path, "r") as f:
    lines = f.read()
    for line in lines.split("\n"):
        if line == "\n" or line == "":
            continue
        # if line.startswith(" ") or line.startswith("-"):
        #     line = line.lstrip("-").lstrip(" ")
        val_med_query_f_metamap_list.append(line.lower().replace(" ", "").replace("?", "").replace("\n", "").replace("-", "").replace("'", "").replace("\"", "").replace(",", "").replace(".", "").replace("/", "").replace(">", ""))
        #val_med_query_f_metamap_list.append(line.strip(" ").strip("\n").replace("  ", " "))
f.close()
print(len(test_med_query_f_metamap_list), len(val_med_query_f_metamap_list))

246 569


In [14]:
test_queries = {'med':[], 'nonmed': []}
val_queries = {'med':[], 'nonmed': []}
med_total, nonmed_total = 0, 0
for key, value in test[0].items():
    vid = next(iter(value))
    ismed = False
    newkey = key.lower().replace(" ", "").replace("?", "").replace("\n", "").replace("-", "").replace("'", "").replace("\"", "").replace(">", "").replace(",", "").replace(".", "").replace("/", "")
    if newkey  in test_med_query_f_metamap_list:
        ismed = True
    if ismed:
        med_total += 1
        test_queries['med'].append([key, vid.replace(".mp4", ""), value[vid]['bounds']])
    else:
        nonmed_total += 1
        test_queries['nonmed'].append([key, vid.replace(".mp4", ""), value[vid]['bounds']])
    
print(med_total, nonmed_total)
med_total, nonmed_total = 0, 0
for key, value in val[0].items():
    vid = next(iter(value))
    ismed = False
    newkey = key.lower().replace(" ", "").replace("?", "").replace("\n", "").replace("-", "").replace("'", "").replace("\"", "").replace(">", "").replace(",", "").replace(".", "").replace("/", "")
    if newkey  in val_med_query_f_metamap_list:
        ismed = True
    if ismed:
        med_total += 1
        val_queries['med'].append([key, vid.replace(".mp4", ""), value[vid]['bounds']])
    else:
        nonmed_total += 1
        val_queries['nonmed'].append([key, vid.replace(".mp4", ""), value[vid]['bounds']])

print(med_total, nonmed_total)

246 7372
569 14675


In [15]:
vid_duration_json = "/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa/video_duration.json"
video_duration_dict = load_jsonl(vid_duration_json)[0]

In [16]:
# randomly sample 200 from medical queries and 200 from non-medical queries
import random

test_med_len = len(test_queries['med'])
test_nonmed_len = len(test_queries['nonmed'])
val_med_len = len(val_queries['med'])
val_nonmed_len = len(val_queries['nonmed'])

random.seed(9)
test_med_idx = random.sample(list(range(0, test_med_len)), k = 200)
test_nonmed_idx = random.sample(list(range(0, test_nonmed_len)), k = 200)
val_med_idx = random.sample(list(range(0, val_med_len)), k = 200)
val_nonmed_idx = random.sample(list(range(0, val_nonmed_len)), k = 200)
test_med_sampled_queries, test_nonmed_sampled_queries, val_med_sampled_queries, val_nonmed_sampled_queries = {}, {}, {}, {}

# create json file
# if one video have different queries, change video name (append v1, v2, ...)

videos = []
cnt = 0
for idx in test_med_idx:
    qa_dict = {}
    video_id = test_queries['med'][idx][1] + ".mp4"
    src_asr_file = os.path.join("/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/tvqa/ASR", test_queries['med'][idx][1] + ".srt")
    src_vid_file = os.path.join("/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/tvqa/eva_clip_features_corrected", video_id + ".pt")

    if video_id not in videos:
        videos.append(video_id)
    else:
        video_id = test_queries['med'][idx][1] + "_v2" + ".mp4"
    qa_dict[video_id] = {}
    qa_dict[video_id]['relevant'] = True
    qa_dict[video_id]['clip'] = True
    qa_dict[video_id]['bounds'] = test_queries['med'][idx][2]
    qa_dict[video_id]['steps'] = []
    qa_dict[video_id]['v_duration'] = video_duration_dict[test_queries['med'][idx][1] + ".mp4"]
    test_med_sampled_queries[test_queries['med'][idx][0]] = qa_dict


videos = []
cnt = 0
for idx in test_nonmed_idx:
    qa_dict = {}
    video_id = test_queries['nonmed'][idx][1] + ".mp4"
    src_asr_file = os.path.join("/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/tvqa/ASR", test_queries['nonmed'][idx][1] + ".srt")
    src_vid_file = os.path.join("/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/tvqa/eva_clip_features_corrected", video_id + ".pt")

    if video_id not in videos:
        videos.append(video_id)
    else:
        video_id = test_queries['nonmed'][idx][1] + "_v2" + ".mp4"
        
    qa_dict[video_id] = {}
    qa_dict[video_id]['relevant'] = True
    qa_dict[video_id]['clip'] = True
    qa_dict[video_id]['bounds'] = test_queries['nonmed'][idx][2]
    qa_dict[video_id]['steps'] = []
    qa_dict[video_id]['v_duration'] = video_duration_dict[test_queries['nonmed'][idx][1] + ".mp4"]
    test_nonmed_sampled_queries[test_queries['nonmed'][idx][0]] = qa_dict


videos = []
cnt = 0
for idx in val_med_idx:
    qa_dict = {}
    video_id = val_queries['med'][idx][1] + ".mp4"
    src_asr_file = os.path.join("/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/tvqa/ASR", val_queries['med'][idx][1] + ".srt")
    src_vid_file = os.path.join("/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/tvqa/eva_clip_features_corrected", video_id + ".pt")
    
    if video_id not in videos:
        videos.append(video_id)  
    else:
        video_id = val_queries['med'][idx][1] + "_v2" + ".mp4"
   
    qa_dict[video_id] = {}
    qa_dict[video_id]['relevant'] = True
    qa_dict[video_id]['clip'] = True
    qa_dict[video_id]['bounds'] = val_queries['med'][idx][2]
    qa_dict[video_id]['steps'] = []
    qa_dict[video_id]['v_duration'] = video_duration_dict[val_queries['med'][idx][1] + ".mp4"]
    val_med_sampled_queries[val_queries['med'][idx][0]] = qa_dict


videos = []
cnt = 0
for idx in val_nonmed_idx:
    qa_dict = {}
    video_id = val_queries['nonmed'][idx][1] + ".mp4"
    src_asr_file = os.path.join("/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/tvqa/ASR", val_queries['nonmed'][idx][1] + ".srt")
    src_vid_file = os.path.join("/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/tvqa/eva_clip_features_corrected", video_id + ".pt")
    
    if video_id not in videos:
        videos.append(video_id)
    else:
        video_id = val_queries['nonmed'][idx][1] + "_v2" + ".mp4"

    qa_dict[video_id] = {}
    qa_dict[video_id]['relevant'] = True
    qa_dict[video_id]['clip'] = True
    qa_dict[video_id]['bounds'] = val_queries['nonmed'][idx][2]
    qa_dict[video_id]['steps'] = []
    qa_dict[video_id]['v_duration'] = video_duration_dict[val_queries['nonmed'][idx][1] + ".mp4"]
    val_nonmed_sampled_queries[val_queries['nonmed'][idx][0]] = qa_dict



In [23]:
for folder in folders:
    print(f"{folder}\n")

    test_result_json_path = f"{folder}/all_data_test_moment_retrieval_BEST_with_audio.json"
    val_result_json_path = f"{folder}/all_data_val_moment_retrieval_BEST_with_audio.json"



    test_result = load_result_json(test_result_json_path)
    val_result = load_result_json(val_result_json_path)

    med_iou_thresh_3, med_iou_thresh_5, med_iou_thresh_7, med_cnt = 0, 0, 0, 0
    nonmed_iou_thresh_3, nonmed_iou_thresh_5, nonmed_iou_thresh_7, nonmed_cnt, cnt = 0, 0, 0, 0, 0
    for key, value in test[0].items():
        if key in test_result and (key in list(test_nonmed_sampled_queries.keys()) or key in list(test_med_sampled_queries.keys())):
            #print(key)
            vid = next(iter(value))
            iou = compute_iou(value[vid]['bounds'], test_result[key][vid]['bounds'])
            cnt += 1
            if key in list(test_med_sampled_queries.keys()): 
                med_cnt += 1
                if iou >= 0.3:
                    med_iou_thresh_3 += 1 
                if iou >= 0.5:
                    med_iou_thresh_5 += 1 
                if iou >= 0.7:
                    med_iou_thresh_7 += 1
            else:
                nonmed_cnt += 1
                if iou >= 0.3:
                    nonmed_iou_thresh_3 += 1 
                if iou >= 0.5:
                    nonmed_iou_thresh_5 += 1 
                if iou >= 0.7:
                    nonmed_iou_thresh_7 += 1
    print(f"Med TEST IoU=0.3: {med_iou_thresh_3/(med_cnt) * 100}\nIoU=0.5: {med_iou_thresh_5/(med_cnt) * 100}\nIoU=0.7: {med_iou_thresh_7/(med_cnt) * 100}\ntotal queries:{med_cnt}/ {cnt}")
    print(f"\nNonmed TEST IoU=0.3: {nonmed_iou_thresh_3/(nonmed_cnt) * 100}\nIoU=0.5: {nonmed_iou_thresh_5/(nonmed_cnt) * 100}\nIoU=0.7: {nonmed_iou_thresh_7/(nonmed_cnt) * 100}\ntotal queries:{nonmed_cnt} / {cnt}")


    med_iou_thresh_3, med_iou_thresh_5, med_iou_thresh_7, med_cnt = 0, 0, 0, 0
    nonmed_iou_thresh_3, nonmed_iou_thresh_5, nonmed_iou_thresh_7, nonmed_cnt, cnt = 0, 0, 0, 0, 0

    for key, value in val[0].items():
        if key in val_result and (key in list(val_nonmed_sampled_queries.keys()) or key in list(val_med_sampled_queries.keys())):
            vid = next(iter(value))
            iou = compute_iou(value[vid]['bounds'], val_result[key][vid]['bounds'])
            cnt += 1
            if key in list(val_med_sampled_queries.keys()):
                med_cnt += 1
                if iou >= 0.3:
                    med_iou_thresh_3 += 1 
                if iou >= 0.5:
                    med_iou_thresh_5 += 1 
                if iou >= 0.7:
                    med_iou_thresh_7 += 1 
            else:
                nonmed_cnt += 1
                if iou >= 0.3:
                    nonmed_iou_thresh_3 += 1 
                if iou >= 0.5:
                    nonmed_iou_thresh_5 += 1 
                if iou >= 0.7:
                    nonmed_iou_thresh_7 += 1
    print(f"\nMed Val IoU=0.3: {med_iou_thresh_3/(med_cnt) * 100}\nIoU=0.5: {med_iou_thresh_5/(med_cnt) * 100}\nIoU=0.7: {med_iou_thresh_7/(med_cnt) * 100}\ntotal queries:{med_cnt}/ {cnt}")
    print(f"\nNonmed Val IoU=0.3: {nonmed_iou_thresh_3/(nonmed_cnt) * 100}\nIoU=0.5: {nonmed_iou_thresh_5/(nonmed_cnt) * 100}\nIoU=0.7: {nonmed_iou_thresh_7/(nonmed_cnt) * 100}\ntotal queries:{nonmed_cnt} / {cnt}")


    print("\n\n")

/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa

Med TEST IoU=0.3: 5.0
IoU=0.5: 3.0
IoU=0.7: 2.0
total queries:200/ 400

Nonmed TEST IoU=0.3: 8.0
IoU=0.5: 5.0
IoU=0.7: 2.5
total queries:200 / 400

Med Val IoU=0.3: 9.0
IoU=0.5: 4.0
IoU=0.7: 2.0
total queries:200/ 400

Nonmed Val IoU=0.3: 15.0
IoU=0.5: 4.0
IoU=0.7: 2.5
total queries:200 / 400



/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa_finetuned_versions/tvqa_finetuned_on_visual_med

Med TEST IoU=0.3: 11.0
IoU=0.5: 3.5000000000000004
IoU=0.7: 1.5
total queries:200/ 400

Nonmed TEST IoU=0.3: 12.5
IoU=0.5: 5.5
IoU=0.7: 1.5
total queries:200 / 400

Med Val IoU=0.3: 11.0
IoU=0.5: 3.5000000000000004
IoU=0.7: 0.5
total queries:200/ 400

Nonmed Val IoU=0.3: 7.5
IoU=0.5: 4.0
IoU=0.7: 1.5
total queries:200 / 400



/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa_finetuned_versions/tvqa_finetuned_on_visual_nonmed

Med TEST IoU=0.3: 6.0
IoU=0.5: 3.5000000000000004
IoU=0.7: 1.0
total queries:200/ 400
